# This one fixes one component to the same as LLL
## IF the LLL has a (... 1, ...) somewhere

beru to tak, že mám >tries< matic, které mají někde v LLLku jedničku a na té jedničce to zkoušim

In [206]:
# from closest_vs_minimal import shortest_lc_in_cube
# from copy import deepcopy
%run closest_versus_minimal_tools.ipynb

In [207]:
# UNINTERESTING TOOLS
from random import randint, seed

def shortestVector(matrix): 
    """
    :returns: 
        - n - norm of the shortest vector
        - v - the shortest vector (vector)
        - i - index of the SV
    """
    return sorted([(matrix[idx].norm().n(), matrix[idx], idx) for idx in range(matrix.nrows())])[0][1]

def randomMatrix(dimension, per) -> matrix:
    '''
    Returns a random square matrix with full rank.
    INPUT:
    dimension: dimension of the random matrix
    per: perimeter of the components
    '''
    list = [randint(-per, per) for _ in range(dimension**2)]
    M = matrix(ZZ, dimension, dimension, list)
    while M.rank() != dimension:
        list = [randint(-per, per) for _ in range(dimension**2)]
        M = matrix(ZZ, dimension, dimension, list)
    return M

def gram_matrix(matrix) -> matrix:
    return matrix * matrix.transpose()

def matrix_to_list(A) -> list:
    return [[int(A.list()[row * A.ncols() + col]) for col in range(A.ncols())] for row in range(A.nrows())]

def vector_to_list(vector) -> list:
    return [float(num) for num in vector.list()]

### Interesting math tools. 

Following functions actually have some thoughts behind them. 


`abnormality_data`
Given a matrix $B$ and it's shortest vector given by the LLL algorithm, returns __a dictionary__ with data concerting the SV given by the cube algorithm, if it's far enough from the LLL solution. 

`find_real_minimum` Given a matrix $B$ and it's shortest vector given by the LLL algorithm, returns the __linear combination of the SV given by the cube algorithm__.




### Formatting tools

These functions provide transfering the data to the output jsonfile as described above.

In [208]:
### output formatting
import json
import numpy as np

def format_data(output_data):
    for dic in output_data:
        for key, value in dic.items():
            if isinstance(value, sage.matrix.matrix_integer_dense.Matrix_integer_dense):
                dic[key] = matrix_to_list(value)
            elif isinstance(value, (int, float, sage.rings.real_mpfr.RealNumber)):
                dic[key] = float(value) 
            else:
                dic[key] = vector_to_list(value)
    return output_data

    
def into_json(data, jsonfilename):
    with open(jsonfilename, 'w') as f:
        json.dump(data, f)

def from_json(filename):
    out_file = open(filename)
    return json.load(out_file)

def print_listdict(list) -> None:
    """
    :param list: list of dictionaries
    """
    for dictionary in list:
        for pair in dictionary.items():
            print(pair[1], ": ", pair[0])
        print()

def norm_G(lin_comb, Gram_matrix):
    tmp = [sum(x * y for x, y in zip(lin_comb, col)) for col in zip(*Gram_matrix)]
    return sum(x * y for x, y in zip(tmp, lin_comb))
    
def into_dict(B, lcLLL: vector, lcCube: int, lc_exact) -> dict:
    result = {}
    G = gram_matrix(B)
    lc_shortest_in_cube=  vector(shortest_lc_in_cube(lcCube, B))
    result["B"] = B
    result["G"] = G
    result["lincomb_exact"] = lc_exact
    result["lincomb_LLL"] = lcLLL
    result["lincomb_cube"] = lcCube
    result["lincomb_cube_shortest"] = lc_shortest_in_cube
    result["exact.norm"] = vector(lc_exact*B).norm().n(digits=5)
    result["LLL.norm"] = vector(lcLLL*B).norm().n(digits=5)
    result["shortest_cube.norm"] = (vector(lc_shortest_in_cube)*B).norm().n(digits=5)
    result["sv_LLL"] = lcLLL * B
    result["sv_exact"] = lc_exact * B
    result["sv_shortst_cube"] = lc_shortest_in_cube* B
    return result

### Main function

This cell actually generates the new examples and checks wheter a matrix is *dysfunctional*. 

`generate_new_examples`
The main function. Based on input parameters above, generates *some* number of *dysfunctional* matrices, computes their invariants such as Gram matrix, LLL/cube linear combinations etc. and creates a json file with this information (specified above).

`is_dysfunctional`
Given a matrix $B$, checks whether its dysfunctional and if so, returns the data describing the case.


In [209]:
# MAIN 
from sage.modules.free_module_integer import IntegerLattice


def generate_new_examples(iterations, dimension, perimeter, rng, jsonfilename, printing = False, functioning = True) -> None:
    '''
    generates >iterations< of >dis/functioning< matrices and saves them in a json file.
    returns number of suitable cases
    '''
    output_data = []
    i = 0
    # for i in range(iterations):
    while i < iterations:
        B = randomMatrix(dimension, perimeter)
        G = gram_matrix(B)

        L = IntegerLattice(B, lll_reduce=False)

        sv_LLL = shortestVector(B.LLL())
        lcLLL = B.solve_left(sv_LLL)

        if 1 not in lcLLL.list():
            continue

        sv_exact = L.shortest_vector()
        lc_exact = B.solve_left(sv_exact)
        sv_exact_norm = sv_exact.norm().n()
        

        # Compare the result with solution of the cube algorithm
        this_case_works, lcCube = compare_with_cube(lcLLL, G, rng)
        if this_case_works != functioning:
            case_info = into_dict(B, lcLLL, lcCube, lc_exact)
            output_data.append(case_info)
        i +=1
    
    # Format output data
    if printing: print_listdict(output_data)
    dict = format_data(output_data)
    into_json(dict, jsonfilename)
    return len(output_data)


def find_indices(lst, targets):
    return [i for i, x in enumerate(lst) if x in targets]


def compare_with_cube(lcLLL, G, rng) -> bool:
    working_case = None
    for current_row in find_indices(lcLLL, [1,-1]):
        # for K in range(2, rng):
        LLL_component = lcLLL[current_row]
        lcCube = find_real_minimum(G, current_row, LLL_component)
        working_case = lcCube                

        if norm_G(lcCube, G) > norm_G(lcLLL, G):
            print("cube bigger than LLL")
        # Check for invalid cases
        if norm_G(lcCube, G) > norm_G(lcLLL, G) or lcCube == zero_vector(SR, dimension):
            continue

        # Check if the two results are the same
        for i in range(dimension - 1):  
            difference = abs(lcLLL[i]) - abs(lcCube[i])
            if abs(difference) >= sensitivity:
                return True, lcCube
    return False, working_case
        
def find_real_minimum(G, current_row, LLL_component) -> vector:
    """
    returns linear combination of the minimum over real numbers
    the >current_row< parameter is the coordinate of fixed col/row idk
    the >K< parameter is the number inserted into this column
    """
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([LLL_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right((-1) * matrixB)).list()
    result.insert(current_row, LLL_component)
    return vector(result).n(digits=5)

In [211]:
dimension = 10

perimeter = 1000
sensitivity = 1
tries = 100
# jsonfilename = "basic_cube_nonf_matrices_%sx%s_%s_tries_%s_perimeter.json"% (dimension, dimension, tries ,perimeter)
jsonfilename = "dummyfile.json"
range_for_K = 4
generate_new_examples(tries, dimension, perimeter, range_for_K, jsonfilename, printing=True, functioning=False)

[-835 -232 -278  117  479  504 -258    0 -731  218]
[ 483  721  -35 -478 -588   29  175 -241  321  433]
[ -74  376 -292  759  -60 -922 -494   14 -966 -588]
[ 572  893  172 -112  862  246  809 -724  199  793]
[  20 -110 -596 -206  900 -784  415 -261 -352  674]
[ 819  943 -387 -744 -691 -282 -876  182 -417  -41]
[-966  769 -930 -606   32  126 -333 -528  816  342]
[ 258  855 -797 -841 -285  887  378  705 -267 -623]
[-222  272 -928  675 -929  438  532 -924  655 -214]
[  70 -346  776   27  359 -172  380 -141 -774  934] :  B
[ 1973928 -1069216   356523  -390151   483544  -833323   458646   -18249  -427727   565892]
[-1069216  1708660  -763200  1191230  -187882  1494015   873922   905044   791047  -215353]
[  356523  -763200  3185013 -1188624   378742  1005956  -778100  -456537  -228737  -195533]
[ -390151  1191230 -1188624  3817428  1405839  -293631   647122    88941   247653  1124796]
[  483544  -187882   378742  1405839  2653334  -396138   448111  -745593  -712666  1125815]
[ -833323  1494

45

In [ ]:
# dimension = 6
# perimeter = 10000
# sensitivity = 1
# tries = 10000
# jsonfilename = "basic_cube_nonf_matrices_%sx%s_%s_tries_%s_perimeter.json"% (dimension, dimension, tries ,perimeter)
# range_for_K = 4
# generate_new_examples(tries, dimension, perimeter, range_for_K, jsonfilename, printing=False, functioning=False)